# Zero to Cortex  
## *Basics*
This lab will take you through the basics of the Snowflake Cortex LLM offering. You'll have a chance to explore the functions and understand how they work with custom prompts and real data in both SQL and Python.  
https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions

![Alt text](https://venturebeat.com/wp-content/uploads/2024/04/a-robot-playing-with-a-snowflake-in-arctic-cinemat-4OYW23nATBm50aD_slLk8w-xaTFE1EbSLmDWJXvWCxXrA.jpeg?fit=750%2C422&strip=all "a title")

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
from snowflake.snowpark import functions as F
from snowflake.cortex import Complete, Sentiment, Summarize, Translate

### Translate  
https://docs.snowflake.com/en/sql-reference/functions/translate-snowflake-cortex

In [ ]:
-- TRANSLATE
-- Translate text from English to Italian
-- We can leave the input language blank and it will infer
select
    snowflake.cortex.translate(
        'I am using Snowflake Cortex artificial intelligence functions to translate text!',
        '',
        'it');

In [ ]:
# Translate some text from English to Polish
print(
    Translate(
    "I am using Snowflake Cortex artificial intelligence functions to translate text!",
    "",
    "pl"
))

In [ ]:
select
    description,
    snowflake.cortex.translate(
        description,
        '',
        'ko'
    ) as review_translation
from
    wine_reviews
limit 10;

In [ ]:
session.table("wine_reviews").select(
    F.col("description"),
    Translate(
        F.col("description"),
        "",
        "sv"
    ).alias("translation")
).limit(10).toPandas()

### Sentiment 
https://docs.snowflake.com/en/sql-reference/functions/sentiment-snowflake-cortex

In [ ]:
select
    snowflake.cortex.sentiment(
        'This group is really the best. They are so smart!'
    );

In [ ]:
print(Sentiment("I don't really mind the weather, but it isn't my favorite."))

In [ ]:
select
    review,
    snowflake.cortex.sentiment(review) as sentiment
from
    airline_reviews
limit 5;

In [ ]:
session.table("airline_reviews").select(
    F.col("review"),
    Sentiment(F.col("review")).alias("sentiment")
).limit(5).toPandas()

### Summarize  
https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex

In [ ]:
select
    snowflake.cortex.summarize('I had to go to the store to get coffee because we ran out. I didn\'t realize until later that we had a whole closet full of coffee. Could have saved the trip!.');

In [ ]:
print(
    Summarize(
        """I've been drinking bottled water instead of the water from my tap.
        I found out that our house has a private well and the water we have
         in our well is way better. I am going to stop buying bottled water."""
    )
)

In [ ]:
select
    airport_review,
    snowflake.cortex.summarize(airport_review)
from
    airport_reviews
limit 10;

In [ ]:
session.table("airport_reviews").select(
    F.col("airport_review"),
    Summarize(F.col("airport_review")).alias("review_summary")
).limit(5).toPandas()

### COMPLETE  
https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex

In [ ]:
select
    snowflake.cortex.complete(
        'snowflake-arctic',
        'How can we change the world with AI in three sentences?'
    );

In [ ]:
print(
    Complete(
        "snowflake-arctic",
        "What can you tell me about Wheaten Terriers?"
    )
)

In [ ]:
select
    description,
    snowflake.cortex.complete(
        'snowflake-arctic',
        CONCAT(
            'You\'re a wine sommelier. I really like grape juice and I don\'t like dry
            wines. Based on the review between <review> and </review>, can you tell me if 
            this would be a good wine for me? <review>', description, '</review>'
        )
    ) as answer
from
    wine_reviews
limit 2;

In [ ]:
session.table("wine_reviews").select(
    F.col("description"),
    Complete(
        "snowflake-arctic",
        F.concat(
            F.lit("""Based on the value between the <context> and </context> tags, please provide me with an understanding
            of what I should expect if I drink this wine.
            <context>"""),
            F.col("description"),
            F.lit("</context>"))
    ).alias("expectation")
).limit(10).toPandas()

### EMBED_TEXT
https://docs.snowflake.com/sql-reference/functions/embed_text-snowflake-cortex

In [ ]:
select
    snowflake.cortex.embed_text_768(
        'snowflake-arctic-embed-m',
        'California Contemporary style homes'
    );

In [ ]:
select
    description,
    snowflake.cortex.embed_text_768(
        'snowflake-arctic-embed-m',
        description
    ) as description_embedding
from
    wine_reviews
limit 10;

### VECTOR DISTANCE CALCULATIONS  
https://docs.snowflake.com/en/sql-reference/functions/vector_cosine_similarity

In [ ]:
select
    vector_cosine_similarity(
        snowflake.cortex.embed_text_768('snowflake-arctic-embed-m', 'California Contemporary style homes'),
        snowflake.cortex.embed_text_768('snowflake-arctic-embed-m', 'California Contemporary style homes')
    );

In [ ]:
select
    *,
    vector_cosine_similarity(
        information_embeds,
        snowflake.cortex.embed_text_768('snowflake-arctic-embed-m', 'I like wines that are sweet and from California.')
    ) as similarity
from
    wine_reviews
order by
    similarity desc
limit 10;

Feel free to write your own queries below. SQL or Python!